In [1]:
from redcap import Project
import pandas as pd

#REDCap API configuration
api_url = 'https://ruff.mrc.gm:8443/redcap/api/'
api_key = 'D6807C091B1FC14F57CC9B2972D0B398'
project = Project(api_url, api_key)

In [2]:

#extract infant ID
infant_id = project.export_records(forms=['infantid'])
infant_id=pd.DataFrame(infant_id)


infantID_data = infant_id[['con_participantid_q1','infantid','redcap_event_name']]
infantID_data=pd.DataFrame(infantID_data)
infantID_data = infantID_data[
    (infantID_data['redcap_event_name'] == 'baby_check_arm_2') &
    ((infantID_data['infantid'].notnull()))
]

infantID_data = pd.DataFrame(infantID_data)

In [3]:
## ALL DATA THAT ARE IN THE WEEK 28.CSV EXCLUDED
#Read from the ones that complete week 28: "Week 28.csv"
#C:/User/yuceesay/OneDrive - London School of Hygiene and Tropical Medicine/Desktop/INDiGO Trial/Call List/CallLIst_scripts
wk28_uss = pd.read_csv('Week 28.csv')

# Merge wk28_uss to infantID_data on 'participant_id'
wk28_infID_merge = pd.merge(wk28_uss, infantID_data, on= 'con_participantid_q1', how='inner')

# Filter out the common participant IDs
wk28_import = wk28_uss[~wk28_uss['con_participantid_q1'].isin(wk28_infID_merge['con_participantid_q1'])]


In [4]:
##Exclude all withdrawals from wk28_import data.
#Read from the ones that complete withdrawals: "Withdrawals.csv"
df_withdrawals = pd.read_csv('Withdrawals.csv')

# Merge the dataframes on 'participant_id'
wk28_withd_merged = pd.merge(wk28_import, df_withdrawals, on= 'con_participantid_q1', how='inner')

# Filter out the common participant IDs
wk28_import= wk28_import[~wk28_import['con_participantid_q1'].isin(wk28_withd_merged['con_participantid_q1'])]

In [5]:
##MERGE FINAL WK28_IMPORT DATA TO INDiGO_ID data
indigo_id = pd.read_csv('INDiGO_ID.csv')
wk28_id_merged = pd.merge(indigo_id,wk28_import,on='con_participantid_q1', how='inner')

In [6]:
# week 28 uss scan data into a data frame
arm2_id = pd.DataFrame({
    'con_participantid_q1' : wk28_id_merged['con_participantid_q1'],
    'infantid': wk28_id_merged['infantid'],
    'infantid_complete': '2',
    'redcap_event_name': 'baby_check_arm_2'
})

In [7]:
import datetime
# Get today's date and time
now = datetime.datetime.now()

# Format the current date and time
current_datetime = now.strftime('%Y%m%d_%H%M%S')

# Create the filename with the current date and time
filename = f'infantID_import_{current_datetime}.csv'

 #Save the filtered DataFrame to a CSV file
arm2_id.to_csv(filename, index=False)